<a href="https://colab.research.google.com/github/MahdiTheGreat/Game-playing-systems/blob/main/AI_TTT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install -Uqq ipdb
import ipdb

In [18]:
%pdb off

Automatic pdb calling has been turned OFF


In [30]:
import numpy as np
import random
import time
from math import sqrt, log
from copy import deepcopy
from IPython.display import clear_output
from google.colab import output

class MCTSNode:
    def __init__(self, state, symbol,parent=None):
        self.state = state  # Tic-Tac-Toe board state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.value = 0
        self.untried_moves = self.get_legal_moves()
        self.symbol = symbol

    def get_legal_moves(self):
        return [(i, j) for i in range(len(self.state)) for j in range(len(self.state[0])) if self.state[i, j] == ""]

    def best_child(self, heuristic_score, exploration_weight=1.41):
     # Incorporate heuristic score into node selection
     return max(self.children, key=lambda child: (child.value / (child.visits + 1e-6) +
                                                  exploration_weight * sqrt(log(self.visits + 1) / (child.visits + 1e-6)) +
                                                 heuristic_score(state=child.state,symbol=self.symbol)))

    def expand(self):
        if not self.untried_moves:
            return self  # Return self if no moves left to expand
        move = self.untried_moves.pop()
        #new_state = self.state.copy()
        new_state = deepcopy(self.state)
        #new_state[move] = "X" if np.count_nonzero(self.state == "X") <= np.count_nonzero(self.state == "O") else "O"
        new_state[move] = self.symbol
        child_node = MCTSNode(state=new_state,symbol=self.symbol,parent=self)
        self.children.append(child_node)
        return child_node

    def update(self, result):
        self.visits += 1
        self.value += result


def default_heuristic_score(state, symbol):
  return 0

def blocker_heuristic_score(state,symbol):

      def is_threatening_win(state, x, y, symbol):
        size = len(state)
        # Horizontal, vertical, and diagonal win checks
        win_conditions = [
            all(state[x][i] == symbol for i in range(size)),  # Row check
            all(state[i][y] == symbol for i in range(size)),  # Column check
            x == y and all(state[i][i] == symbol for i in range(size)),  # Major diagonal check
            x + y == size - 1 and all(state[i][size - 1 - i] == symbol for i in range(size))  # Minor diagonal check
        ]
        return any(win_conditions)

      score = 0
      size = len(state)
      opponent = "O" if symbol == "X" else "X"

      # Scoring for center and corners as before
      centers = [(size//2, size//2)] if size % 2 == 1 else [(size//2 - 1, size//2 - 1), (size//2 - 1, size//2), (size//2, size//2 - 1), (size//2, size//2)]
      corners = [(0, 0), (0, size - 1), (size - 1, 0), (size - 1, size - 1)]
      for center in centers:
          if state[center] == symbol:
              score += 4
      for corner in corners:
          if state[corner] == symbol:
              score += 2

      # Checking for immediate threats to block
      for i in range(size):
          for j in range(size):
              if state[i][j] == "":
                  # Check horizontal, vertical, and diagonal lines for threats
                  state[i][j] = symbol  # Temporarily make the move
                  if is_threatening_win(state, i, j, symbol):
                      score += 3  # Adding score for blocking potential wins
                  state[i][j] = opponent
                  if is_threatening_win(state, i, j, opponent):
                      score += 3  # Adding score for preventing opponent's immediate wins
                  state[i][j] = ""  # Undo the move

      return 0.1*score

class MCTS:
    def __init__(self, symbol,heuristic_score):
     self.symbol = symbol
     self.heuristic_score = heuristic_score

    def search(self, root):
        start_time = time.time()
        iterations = 0

        #if root.untried_moves:
        #    root = root.expand()  # Ensure root expands at least once

        #while time.time() - start_time < self.time_limit and iterations < self.max_iterations:
        node = self.select(root)
        #if node.untried_moves:
        node = node.expand()
        result = self.simulate(node.state)
        self.backpropagate(node, result)
        iterations += 1

        #return max(root.children, key=lambda c: c.visits, default=root)  # Choose best move by visits
        return root.best_child(heuristic_score=self.heuristic_score)  # Choose best move by value


    def select(self, node):
     while node.children and not node.untried_moves:
         node = node.best_child(heuristic_score=self.heuristic_score)
     return node


    def check_winner(self, state):
     for player in ["X", "O"]:
      for i in range(len(state)):
          if all(state[i, :] == player) or all(state[:, i] == player):
              return player
      if all(state.diagonal() == player) or all(np.fliplr(state).diagonal() == player):
          return player
     return None

    def find_best_action(self, state, symbol):
     size = len(state)
     best_score = float('-inf')
     best_move = None
     # Iterate over all empty cells
     for i in range(size):
         for j in range(size):
             if state[i][j] == "":
                 # Simulate placing the player's symbol
                 state[i][j] = self.symbol
                 score = self.heuristic_score(state=state, symbol=symbol)
                 state[i][j] = ""  # Undo move

                 # Choose the move with the highest heuristic score
                 if score > best_score:
                     best_score = score
                     best_move = (i, j)
     return best_move

    def random_move(self, state):
        size = len(state)
        legal_moves = [(i, j) for i in range(size) for j in range(size) if state[i, j] == ""]
        if not legal_moves:
            return None
        return random.choice(legal_moves)

    def simulate(self, state):
        state = deepcopy(state)
        current_player = self.symbol
        while True:
            if self.heuristic_score == default_heuristic_score:
                move = self.random_move(state)
            else:
                move = self.find_best_action(state=state, symbol=current_player)
            state[move] = current_player
            winner = self.check_winner(state)
            if winner == current_player:
                return 1 if current_player == "X" else -1
            current_player = "O" if current_player == "X" else "X"

    def backpropagate(self, node, result):
        while node:
            node.update(result)
            result = -result  # Alternate win/loss perspective
            node = node.parent

class TicTacToe:
    def __init__(self, heuristic_score,human_player="X", current_player="X", board_shape=3):
        self.state = np.full((board_shape, board_shape), "")
        self.symbols = ["X", "O"]
        if human_player in self.symbols:
            self.human_player = human_player
            self.ai_player2 = list(set(self.symbols) - {human_player})[0]
        else:
            self.ai_player = "X"
            self.ai_mcts = MCTS(symbol=self.ai_player,heuristic_score=heuristic_score)
            self.ai_player2 = "O"
            self.human_player = None

        self.current_player = current_player
        self.ai2_mcts = MCTS(symbol=self.ai_player2,heuristic_score=heuristic_score)

    def display_board(self):
            size = len(self.state)

            # Print column numbers
            print("  " + " ".join(str(i) for i in range(size)))

            for i in range(size):
                # Print row number and row content
                print(f"{i} " + "|".join(self.state[i]))

                # Print row separators (except after the last row)
                if i < size - 1:
                    print("  " + "-+" * (size - 1) + "-")

            print()

    def get_human_move(self):
     while True:
      try:
          row = int(input(f"Enter row (0-{len(self.state)-1}): "))
          col = int(input(f"Enter column (0-{len(self.state[0])-1}): "))
          if self.state[row, col] == "":
              self.state[row, col] = self.human_player
              self.current_player = self.ai_player2  # Switch turn
              break
          else:
              print("Invalid move, try again!")
      except (ValueError, IndexError):
          print("Invalid input!")

    def get_ai_move(self,mcts):
     # AI's turn (MCTS)
     print(f"AI({mcts.symbol}) is thinking...")
     #time.sleep(1)
     root = MCTSNode(state=deepcopy(self.state),symbol=mcts.symbol)
     best_move = mcts.search(root)
     self.state = best_move.state
     self.current_player = list(set(self.symbols) - {mcts.symbol})[0]

    def play_game(self):
        while True:
            self.display_board()

            def player_one_ai_player():
              self.get_ai_move(self.ai_mcts)

            player_one=self.human_player if self.human_player else self.ai_player
            player_one_move = self.get_human_move if self.human_player else player_one_ai_player

            if self.current_player == player_one:
              player_one_move()
            else:
              self.get_ai_move(self.ai2_mcts)

            winner=self.ai2_mcts.check_winner(self.state)
            if winner:
                self.display_board()
                print(f"{winner} wins!")
                break
            elif "" not in self.state:
                self.display_board()
                print("It's a draw!")
                break


if __name__ == "__main__":
    board_shape = int(input("Enter board size (e.g., 3 for a 3x3 board): "))
    human_player = input("Choose your player (X or O otherwise self-play): ")
    heuristic_score = blocker_heuristic_score if input("Do you want to use a heuristic? (y/n): ").lower() == "y" else default_heuristic_score
    game = TicTacToe(human_player=human_player, current_player=human_player, board_shape=board_shape,heuristic_score=heuristic_score)
    game.play_game()



Enter board size (e.g., 3 for a 3x3 board): 3
Choose your player (X or O otherwise self-play): d
Do you want to use a heuristic? (y/n): y
  0 1 2
0 ||
  -+-+-
1 ||
  -+-+-
2 ||

AI(O) is thinking...
  0 1 2
0 ||
  -+-+-
1 ||
  -+-+-
2 ||O

AI(X) is thinking...
  0 1 2
0 ||
  -+-+-
1 ||
  -+-+-
2 |X|O

AI(O) is thinking...
  0 1 2
0 ||
  -+-+-
1 ||
  -+-+-
2 O|X|O

AI(X) is thinking...
  0 1 2
0 ||
  -+-+-
1 ||X
  -+-+-
2 O|X|O

AI(O) is thinking...
  0 1 2
0 ||
  -+-+-
1 |O|X
  -+-+-
2 O|X|O

AI(X) is thinking...
  0 1 2
0 ||
  -+-+-
1 X|O|X
  -+-+-
2 O|X|O

AI(O) is thinking...
  0 1 2
0 ||O
  -+-+-
1 X|O|X
  -+-+-
2 O|X|O

O wins!
